In [1]:
import nest_asyncio
import asyncio
import aiohttp
import random
import pandas as pd
import time
import os

# Apply nest_asyncio to allow asyncio in Jupyter
nest_asyncio.apply()

# API Endpoints
BINANCE_PRICE_URL = "https://api.binance.com/api/v3/ticker/price"
BYBIT_PRICE_URL = "https://api.bybit.com/v5/market/tickers"

# Track last arbitrage detection time per coin
last_arbitrage_time = {}

# Paper Trade Log File
PAPER_TRADE_LOG_FILE = "paper_trade_arbitrage.csv"

async def fetch_price(session, url, params):
    """Fetches the latest price from an API asynchronously."""
    try:
        async with session.get(url, params=params, timeout=5) as response:
            data = await response.json()
            return data
    except Exception as e:
        print(f"⚠️ Error fetching price from {url} with params {params}: {e}")
        return None

async def get_binance_price(session, symbol):
    """Fetches the latest price for a given symbol from Binance asynchronously."""
    data = await fetch_price(session, BINANCE_PRICE_URL, {"symbol": symbol})
    if data and "price" in data:
        return float(data["price"])
    return None

async def get_bybit_price(session, symbol):
    """Fetches the latest price for a given symbol from Bybit asynchronously."""
    data = await fetch_price(session, BYBIT_PRICE_URL, {"category": "spot", "symbol": symbol})
    if data and "result" in data and "list" in data["result"] and len(data["result"]["list"]) > 0:
        return float(data["result"]["list"][0]["lastPrice"])
    return None

def simulate_execution_time():
    """Simulates trade execution time (5 to 30 seconds delay)."""
    return random.randint(5, 30)

async def paper_trade_arbitrage(session, symbol, fees, trade_amount=1000, min_profit=1):
    """Simulates paper trading arbitrage for a given symbol asynchronously."""
    print(f"🔍 Checking arbitrage for {symbol}...")

    binance_price, bybit_price = await asyncio.gather(
        get_binance_price(session, symbol),
        get_bybit_price(session, symbol)
    )

    if binance_price is None or bybit_price is None:
        print(f"⚠️ Skipping {symbol} due to missing price data.")
        return

    # Extract fee info
    binance_taker_fee = fees.get(symbol, {}).get("Taker Fee (Binance)", 0)
    bybit_taker_fee = fees.get(symbol, {}).get("Taker Fee (Bybit)", 0)
    binance_withdraw_fee = fees.get(symbol, {}).get("Withdrawal Fee (Binance)", 0)
    bybit_withdraw_fee = fees.get(symbol, {}).get("Withdrawal Fee (Bybit)", 0)

    # Amount of asset that can be bought with the trade amount
    buy_binance_units = trade_amount / binance_price
    buy_bybit_units = trade_amount / bybit_price

    # Direction 1: Buy on Binance, Sell on Bybit
    buy_binance_sell_bybit_profit = (
        (buy_binance_units * bybit_price) * (1 - bybit_taker_fee)
        - (trade_amount * (1 + binance_taker_fee))
        - (binance_withdraw_fee * bybit_price)
    )

    # Direction 2: Buy on Bybit, Sell on Binance
    buy_bybit_sell_binance_profit = (
        (buy_bybit_units * binance_price) * (1 - binance_taker_fee)
        - (trade_amount * (1 + bybit_taker_fee))
        - (bybit_withdraw_fee * binance_price)
    )

    profitable_trade = None
    trade_direction = None

    if buy_binance_sell_bybit_profit > min_profit:
        profitable_trade = buy_binance_sell_bybit_profit
        trade_direction = "Binance → Bybit"

    if buy_bybit_sell_binance_profit > min_profit:
        if profitable_trade is None or buy_bybit_sell_binance_profit > profitable_trade:
            profitable_trade = buy_bybit_sell_binance_profit
            trade_direction = "Bybit → Binance"

    if profitable_trade:
        execution_time = simulate_execution_time()
        print(f"✅ PROFIT FOUND: {symbol} | {trade_direction} | Profit: ${profitable_trade:.2f}")

        # Log arbitrage details
        await log_paper_trade(symbol, binance_price, bybit_price, trade_direction, profitable_trade, execution_time, trade_amount)

async def log_paper_trade(symbol, binance_price, bybit_price, trade_direction, profit, execution_time, trade_amount):
    """Logs paper trade arbitrage to a CSV file asynchronously."""
    data = {
        "Symbol": symbol,
        "Binance Price": binance_price,
        "Bybit Price": bybit_price,
        "Trade Direction": trade_direction,
        "Profit": profit,
        "Execution Time (s)": execution_time,
        "Trade Amount (USD)": trade_amount,
        "Timestamp": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    }

    file_exists = os.path.exists(PAPER_TRADE_LOG_FILE)
    df = pd.DataFrame([data])
    df.to_csv(PAPER_TRADE_LOG_FILE, mode="a", header=not file_exists, index=False)
    print(f"📌 Logged trade: {symbol} - Profit: ${profit:.2f}")

async def run_multiple_paper_trades(symbols, fees, trade_amount=1000):
    """Runs multiple paper trade arbitrage strategies asynchronously in an infinite loop."""
    async with aiohttp.ClientSession() as session:
        while True:  # Loop forever
            tasks = [paper_trade_arbitrage(session, symbol, fees, trade_amount) for symbol in symbols]
            await asyncio.gather(*tasks)

            print("🔄 Waiting for next arbitrage check...")
            await asyncio.sleep(2)  # Avoid API bans by waiting 2 seconds


from get_metadata import get_metadata

metadata = get_metadata()
# Example symbols to monitor (ensure these exist on both Binance & Bybit)
metadata = metadata[metadata['24h Volume (Binance)'] > 1000000]  # Filter out low volume coins
metadata = metadata[metadata['24h Volume (Bybit)'] > 1000000]  # Filter out low volume coins

# Blacklisted USDT Trading Pairs (Binance & Bybit)
blacklisted_symbols = [
    "USDT", "FDUSDUSDT", "TUSDUSDT", "USDPUSDT", "DAIUSDT",
    "AEURUSDT", "USTUSDT", "USTCUSDT", "PAXGUSDT",
    "BNTUSDT", "PAXGUSDT", "TUSDT", "KMONUSDT", "ZKFUSDT",
    "AURYUSDT", "MFERUSDT", "ZAPUSDT", "BLOCKUSDT", "MELOSUSDT",
    "PARAMUSDT", "GCAKEUSDT", "QMALLUSDT", "DAPPUSDT", "ZECUSDT", "DASHUSDT", "NEIROUSDT", 'TSTUSDT'
]

metadata = metadata[~metadata['Symbol'].isin(blacklisted_symbols)]
symbols_to_monitor = metadata["Symbol"].tolist()
print(f"Monitoring the following symbols: {symbols_to_monitor}")

fees_data = metadata.set_index("Symbol").to_dict(orient="index")

# Run in Jupyter Notebook (This will run indefinitely)
await run_multiple_paper_trades(symbols_to_monitor, fees_data, 10000)


Loading recent metadata...
Monitoring the following symbols: ['1INCHUSDT', 'ACHUSDT', 'AEVOUSDT', 'ALTUSDT', 'ANIMEUSDT', 'ANKRUSDT', 'APEUSDT', 'ARBUSDT', 'ARKMUSDT', 'AXLUSDT', 'BLURUSDT', 'BOMEUSDT', 'BONKUSDT', 'C98USDT', 'CAKEUSDT', 'CGPTUSDT', 'CHZUSDT', 'COOKIEUSDT', 'CRVUSDT', 'DOTUSDT', 'EIGENUSDT', 'ENAUSDT', 'ENJUSDT', 'ETHFIUSDT', 'FETUSDT', 'FIDAUSDT', 'FLOKIUSDT', 'GALAUSDT', 'GMTUSDT', 'GRTUSDT', 'GUSDT', 'HFTUSDT', 'HOOKUSDT', 'HOTUSDT', 'IDUSDT', 'IMXUSDT', 'IOUSDT', 'JASMYUSDT', 'JTOUSDT', 'JUPUSDT', 'LAYERUSDT', 'LDOUSDT', 'LEVERUSDT', 'LRCUSDT', 'MAGICUSDT', 'MANAUSDT', 'MEMEUSDT', 'MOVEUSDT', 'OPUSDT', 'PENGUUSDT', 'PEOPLEUSDT', 'PEPEUSDT', 'PNUTUSDT', 'POLUSDT', 'PORTALUSDT', 'PYTHUSDT', 'RDNTUSDT', 'RENDERUSDT', 'SANDUSDT', 'SHIBUSDT', 'SLPUSDT', 'SOLUSDT', 'SOLVUSDT', 'SPELLUSDT', 'STGUSDT', 'STRKUSDT', 'SUNUSDT', 'SUSHIUSDT', 'TNSRUSDT', 'TRUMPUSDT', 'TRXUSDT', 'USDCUSDT', 'VANRYUSDT', 'WIFUSDT', 'WLDUSDT', 'WOOUSDT', 'WUSDT', 'XAIUSDT', 'XLMUSDT', 'XRPUSDT', '

CancelledError: 

In [ ]:
import time
import csv
import requests
import hmac
import hashlib
import json
import os
from datetime import datetime


with open('keys.json', 'r') as f:
    keys = json.load(f)

BINANCE_API_KEY, BINANCE_API_SECRET = keys['binance']['apiKey'], keys['binance']['secret']
BYBIT_API_KEY, BYBIT_API_SECRET = keys['bybit']['apiKey'], keys['bybit']['secret']

LOG_FILE = "deposit_times.csv"

def get_bybit_deposits():
    """Fetch recent Bybit deposits using API."""
    url = "https://api.bybit.com/v5/asset/deposit/query-record"
    params = {
        "api_key": BYBIT_API_KEY,
        "timestamp": str(int(time.time() * 1000)),
        "type": "1"  # 1 for deposit transactions
    }
    
    # Generate signature
    query_string = "&".join(f"{k}={v}" for k, v in sorted(params.items()))
    signature = hmac.new(BYBIT_API_SECRET.encode(), query_string.encode(), hashlib.sha256).hexdigest()
    params["sign"] = signature
    
    response = requests.get(url, params=params)

    print("Bybit Response Status Code:", response.status_code)
    print("Bybit Raw Response:", response.text)  # Print response before parsing

    if response.status_code == 200:
        try:
            return response.json().get("result", [])
        except json.JSONDecodeError:
            print("Error: Unable to parse Bybit JSON response.")
            return []
    else:
        print("Error fetching Bybit deposits:", response.text)
        return []

get_bybit_deposits()

Bybit Response Status Code: 200
Bybit Raw Response: {"retCode":0,"retMsg":"success","result":{"rows":[],"nextPageCursor":""},"retExtInfo":{},"time":1741655113090}


{'rows': [], 'nextPageCursor': ''}